# Stacking Ensemble

## Data

In [46]:
from typing import Dict, Tuple, Union, List
from tqdm import tqdm
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import os, glob, warnings
from itertools import combinations
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss, accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from sklearn.svm import SVC
from catboost import CatBoostClassifier
from sklearn.ensemble import StackingClassifier
import lightgbm as lgb
from sklearn.model_selection import RepeatedStratifiedKFold
import optuna
import sklearn
warnings.filterwarnings('ignore')

In [2]:
train_df=pd.read_csv("../data/cleaned_train.csv")
test_df=pd.read_csv("../data/cleaned_test.csv")

train_df.head()

,Unnamed: 0,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,days_birth,days_employed,work_phone,home_phone,email,occup_type,family_size,begin_month,credit,CODE
0,0,F,N,N,0,202500.0,Commercial associate,Higher education,Married,Municipal apartment,13899,4709,0,0,0,No job,2.0,6,1.0,F-13899202500.0Commercial associate
1,1,F,N,Y,1,247500.0,Commercial associate,Secondary / secondary special,Civil marriage,House / apartment,11380,1540,0,0,1,Laborers,3.0,5,1.0,F-11380247500.0Commercial associate
2,2,M,Y,Y,0,450000.0,Working,Higher education,Married,House / apartment,19087,4434,0,1,0,Managers,2.0,22,2.0,M-19087450000.0Working
3,3,F,N,Y,0,202500.0,Commercial associate,Secondary / secondary special,Married,House / apartment,15088,2092,0,1,0,Sales staff,2.0,37,0.0,F-15088202500.0Commercial associate
4,4,F,Y,Y,0,157500.0,State servant,Higher education,Married,House / apartment,15037,2105,0,0,0,Managers,2.0,26,2.0,F-15037157500.0State servant


## Preprocessing

### Numerical Data: Scaling

In [7]:
def num_data_scale(train_df, test_df, scaling_cols=None):
    
    scaling_cols = ['child_num','income_total','days_birth','days_employed','family_size','begin_month']

    std_scaler = StandardScaler()
    std_scaler.fit(train_df[scaling_cols])

    train_scaled = std_scaler.transform(train_df[scaling_cols])
    test_scaled = std_scaler.transform(test_df[scaling_cols]) # Apply to the test set using the scaler that was fitted with train data!!

    train_scaled = pd.DataFrame(train_scaled, columns=scaling_cols)
    test_scaled = pd.DataFrame(test_scaled, columns=scaling_cols)

    train_scaled.shape, test_scaled.shape
    
    return train_scaled, test_scaled

In [8]:
train_scaled, test_scaled = num_data_scale(train_df, test_df)

### Categorical Data: One-hot Encoding

In [9]:
def cat_data_encode(train_df, test_df, onehot_cols=None):
    
    onehot_cols = ['gender','car','reality','income_type','edu_type','family_type','house_type','occup_type','work_phone','home_phone','email']
    data = pd.concat([train_df[onehot_cols], test_df[onehot_cols]]) # One-hot encoding using the combination of train and test data
    data = pd.get_dummies(data)

    # data separation
    data = data.reset_index(drop=True)
    train_encoded = data.loc[:train_df.shape[0]-1]
    test_encoded = data.loc[train_df.shape[0]:]
    test_encoded = test_encoded.reset_index(drop=True)
    
    train_encoded.shape, test_encoded.shape
    
    return train_encoded, test_encoded

In [10]:
train_encoded, test_encoded = cat_data_encode(train_df, test_df)

### Merge scaled numerical data with categorical data

In [11]:
def merge_scaled_encoded(train_scaled, test_scaled, train_encoded, test_encoded):
    
    Train = pd.concat([train_scaled,train_encoded],axis=1)
    Train = pd.concat([Train,train_df['credit']],axis=1)

    Test = pd.concat([test_scaled,test_encoded],axis=1)

    Train.shape, Test.shape
    
    return Train, Test

In [12]:
Train, Test = merge_scaled_encoded(train_scaled, test_scaled, train_encoded, test_encoded)

## Split Training and Validation set

In [13]:
x = Train.drop(['credit'], axis=1)
y = Train['credit']

In [14]:
from sklearn.model_selection import train_test_split
train_x, valid_x, train_y, valid_y = train_test_split(x, y, test_size=0.25)

## Model Train


### CatBoost

In [17]:
# FYI: Objective functions can take additional arguments
# (https://optuna.readthedocs.io/en/stable/faq.html#objective-func-additional-args).

from sklearn.metrics import log_loss

def objective(trial):
    param = {
      "random_state":42,
      'learning_rate' : trial.suggest_loguniform('learning_rate', 0.01, 0.3),
      'bagging_temperature' :trial.suggest_loguniform('bagging_temperature', 0.01, 100.00),
      "n_estimators":trial.suggest_int("n_estimators", 1000, 10000),
      "max_depth":trial.suggest_int("max_depth", 4, 16),
      'random_strength' :trial.suggest_int('random_strength', 0, 100),
      "colsample_bylevel":trial.suggest_float("colsample_bylevel", 0.4, 1.0),
      "l2_leaf_reg":trial.suggest_float("l2_leaf_reg",1e-5,3e-3),
      "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
      "max_bin": trial.suggest_int("max_bin", 200, 500),
      'od_type': trial.suggest_categorical('od_type', ['IncToDec', 'Iter']),
  }
    #cat_features = [6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]

    cat = CatBoostClassifier(**param)
    cat.fit(train_x, train_y,
          eval_set=[(train_x, train_y), (valid_x,valid_y)],
          early_stopping_rounds=35,#cat_features=cat_features,
          verbose=100)
    preds = cat.predict_proba(valid_x)
    log_score = log_loss(valid_y, preds)
    # pred_labels = []
    # for i in range(0,len(preds)):
    #     pred_labels.append(np.argmax(preds[i]))
    # accuracy = cal_acc(valid_y_label, pred_labels)
    return log_score

# def cal_acc(true, pred):
#     return np.mean(true==pred)

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=10)

print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2022-12-15 18:14:58,162] A new study created in memory with name: no-name-dc85c19a-eb72-467b-869c-2bbdbb4738d9


0:	learn: 1.0672086	test: 1.0672086	test1: 1.0662014	best: 1.0662014 (0)	total: 67.8ms	remaining: 11m 1s
100:	learn: 0.7375954	test: 0.7375954	test1: 0.8115862	best: 0.8115820 (99)	total: 19.5s	remaining: 31m 6s
200:	learn: 0.5115608	test: 0.5115608	test1: 0.7667450	best: 0.7667450 (200)	total: 54.3s	remaining: 43m 5s


[I 2022-12-15 18:16:14,006] Trial 0 finished with value: 0.7665590837045069 and parameters: {'learning_rate': 0.07498459696609935, 'bagging_temperature': 1.0141741449885584, 'n_estimators': 9766, 'max_depth': 14, 'random_strength': 98, 'colsample_bylevel': 0.45146408420149864, 'l2_leaf_reg': 0.0013874349852284987, 'min_child_samples': 86, 'max_bin': 271, 'od_type': 'Iter'}. Best is trial 0 with value: 0.7665590837045069.


Stopped by overfitting detector  (35 iterations wait)

bestTest = 0.7665590837
bestIteration = 201

Shrink model to first 202 iterations.
0:	learn: 1.0706522	test: 1.0706522	test1: 1.0702351	best: 1.0702351 (0)	total: 18.1ms	remaining: 34.8s
100:	learn: 0.7990077	test: 0.7990077	test1: 0.8171776	best: 0.8171776 (100)	total: 1.45s	remaining: 26.2s
200:	learn: 0.7454025	test: 0.7454025	test1: 0.7912791	best: 0.7912791 (200)	total: 2.83s	remaining: 24.3s
300:	learn: 0.6503473	test: 0.6503473	test1: 0.7662786	best: 0.7662786 (300)	total: 4.62s	remaining: 24.9s
400:	learn: 0.5722687	test: 0.5722687	test1: 0.7572788	best: 0.7572107 (399)	total: 6.47s	remaining: 24.6s


[I 2022-12-15 18:16:22,141] Trial 1 finished with value: 0.7551179672659409 and parameters: {'learning_rate': 0.06308556583612851, 'bagging_temperature': 0.01655546331831464, 'n_estimators': 1923, 'max_depth': 9, 'random_strength': 65, 'colsample_bylevel': 0.5510295128259061, 'l2_leaf_reg': 0.0016472289045823443, 'min_child_samples': 80, 'max_bin': 360, 'od_type': 'IncToDec'}. Best is trial 1 with value: 0.7551179672659409.


...

Stopped by overfitting detector  (35 iterations wait)

bestTest = 0.7576801869
bestIteration = 1618

Shrink model to first 1619 iterations.
0:	learn: 1.0699959	test: 1.0699959	test1: 1.0691036	best: 1.0691036 (0)	total: 4.43ms	remaining: 30.3s
100:	learn: 0.8289677	test: 0.8289677	test1: 0.8197615	best: 0.8197615 (100)	total: 413ms	remaining: 27.6s
200:	learn: 0.8030510	test: 0.8030510	test1: 0.7989665	best: 0.7989665 (200)	total: 828ms	remaining: 27.4s
300:	learn: 0.7846055	test: 0.7846055	test1: 0.7929187	best: 0.7928913 (294)	total: 1.24s	remaining: 26.9s
400:	learn: 0.7699205	test: 0.7699205	test1: 0.7893603	best: 0.7893603 (400)	total: 1.66s	remaining: 26.7s
500:	learn: 0.7561472	test: 0.7561472	test1: 0.7862187	best: 0.7862187 (500)	total: 2.07s	remaining: 26.3s
600:	learn: 0.7440447	test: 0.7440447	test1: 0.7840709	best: 0.7840154 (597)	total: 2.48s	remaining: 25.8s
700:	learn: 0.7331979	test: 0.7331979	test1: 0.7824757	best: 0.7824757 (700)	total: 2.89s	remaining: 25.4s
800:	le

[I 2022-12-15 18:18:26,206] Trial 8 finished with value: 0.776053727549929 and parameters: {'learning_rate': 0.0685586287146963, 'bagging_temperature': 0.016107689845353945, 'n_estimators': 6848, 'max_depth': 4, 'random_strength': 27, 'colsample_bylevel': 0.4235080732783042, 'l2_leaf_reg': 0.002271037583092516, 'min_child_samples': 16, 'max_bin': 259, 'od_type': 'IncToDec'}. Best is trial 4 with value: 0.7539686287809098.


Stopped by overfitting detector  (35 iterations wait)

bestTest = 0.7760537275
bestIteration = 1124

Shrink model to first 1125 iterations.
0:	learn: 1.0881492	test: 1.0881492	test1: 1.0878897	best: 1.0878897 (0)	total: 19.4ms	remaining: 1m 24s
100:	learn: 0.8225873	test: 0.8225873	test1: 0.8314689	best: 0.8314689 (100)	total: 5.95s	remaining: 4m 10s
200:	learn: 0.7766992	test: 0.7766992	test1: 0.8094082	best: 0.8094082 (200)	total: 12.9s	remaining: 4m 26s
300:	learn: 0.7375567	test: 0.7375567	test1: 0.7947204	best: 0.7947204 (300)	total: 20s	remaining: 4m 29s
400:	learn: 0.7063150	test: 0.7063150	test1: 0.7862887	best: 0.7862508 (399)	total: 26.4s	remaining: 4m 19s
500:	learn: 0.6662467	test: 0.6662467	test1: 0.7751208	best: 0.7751208 (500)	total: 34.5s	remaining: 4m 24s
600:	learn: 0.5798445	test: 0.5798445	test1: 0.7597683	best: 0.7597515 (599)	total: 46.3s	remaining: 4m 48s
700:	learn: 0.5085711	test: 0.5085711	test1: 0.7571824	best: 0.7567951 (670)	total: 1m	remaining: 5m 13s


[I 2022-12-15 18:19:27,343] Trial 9 finished with value: 0.7567950619304605 and parameters: {'learning_rate': 0.02434293552638627, 'bagging_temperature': 0.03157187684966995, 'n_estimators': 4349, 'max_depth': 12, 'random_strength': 39, 'colsample_bylevel': 0.8963114454863341, 'l2_leaf_reg': 0.0003202313875545556, 'min_child_samples': 16, 'max_bin': 257, 'od_type': 'IncToDec'}. Best is trial 4 with value: 0.7539686287809098.


Stopped by overfitting detector  (35 iterations wait)

bestTest = 0.7567950619
bestIteration = 670

Shrink model to first 671 iterations.
Number of finished trials: 10
Best trial:
  Value: 0.7539686287809098
  Params: 
    learning_rate: 0.015778207856230707
    bagging_temperature: 0.1872474191063101
    n_estimators: 5431
    max_depth: 11
    random_strength: 2
    colsample_bylevel: 0.4416172828184351
    l2_leaf_reg: 0.0006604407155321971
    min_child_samples: 24
    max_bin: 206
    od_type: Iter


In [18]:
ctb_best = study.best_params
ctb_best

{'learning_rate': 0.015778207856230707,
 'bagging_temperature': 0.1872474191063101,
 'n_estimators': 5431,
 'max_depth': 11,
 'random_strength': 2,
 'colsample_bylevel': 0.4416172828184351,
 'l2_leaf_reg': 0.0006604407155321971,
 'min_child_samples': 24,
 'max_bin': 206,
 'od_type': 'Iter'}

### Random Forest

In [22]:
def objective(trial):

    rf_max_depth = trial.suggest_int("max_depth", 2, 32, log=True)
    classifier_obj = sklearn.ensemble.RandomForestClassifier(
        max_depth=rf_max_depth, n_estimators=10)
    score = sklearn.model_selection.cross_val_score(classifier_obj, train_x, train_y, n_jobs=-1, cv=5, scoring="neg_log_loss")
    log_score = score.mean()
    return log_score

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100)

print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2022-12-15 18:25:12,788] A new study created in memory with name: no-name-59517974-4da2-4450-b5fc-9dc753ddc526
[I 2022-12-15 18:25:13,011] Trial 0 finished with value: -0.8303965578982959 and parameters: {'max_depth': 7}. Best is trial 0 with value: -0.8303965578982959.
[I 2022-12-15 18:25:13,198] Trial 1 finished with value: -0.8449653121070199 and parameters: {'max_depth': 5}. Best is trial 0 with value: -0.8303965578982959.
[I 2022-12-15 18:25:13,366] Trial 2 finished with value: -0.8608352091460677 and parameters: {'max_depth': 2}. Best is trial 0 with value: -0.8303965578982959.
[I 2022-12-15 18:25:13,529] Trial 3 finished with value: -0.8492335878365365 and parameters: {'max_depth': 3}. Best is trial 0 with value: -0.8303965578982959.
[I 2022-12-15 18:25:13,776] Trial 4 finished with value: -0.8080498239122331 and parameters: {'max_depth': 12}. Best is trial 4 with value: -0.8080498239122331.
[I 2022-12-15 18:25:13,970] Trial 5 finished with value: -0.8446476392891082 and para

Number of finished trials: 100
Best trial:
  Value: -0.7940336016880484
  Params: 
    max_depth: 15


In [ ]:
rf_best = study.best_params
rf_best

{'max_depth': 16}

### LightGBM

In [47]:
x_data = Train.drop(['credit'], axis=1)
y_data = Train['credit']

train_y_label = train_y
valid_y_label = valid_y
dtrain = lgb.Dataset(train_x, label=train_y_label)

# FYI: Objective functions can take additional arguments
# (https://optuna.readthedocs.io/en/stable/faq.html#objective-func-additional-args).
def objective(trial):
    param = {
        "objective": "multiclass",
        "metric": "multi_logloss",
        "verbosity": -1,
        "boosting_type": "gbdt",
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
        'num_class': 3,
    }

    # gbm = lgb.train(param, dtrain)
    # preds = gbm.predict(valid_x)
    # log_score = log_loss(valid_y, preds)
    
    model = LGBMClassifier(**param)
    lgb_model = model.fit(train_x, train_y, eval_set=[(valid_x, valid_y)], verbose=0, early_stopping_rounds=25)

    preds = lgb_model.predict_proba(valid_x)
    log_score = log_loss(valid_y, preds)
    
    return log_score

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100)

print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2022-12-15 18:50:57,532] A new study created in memory with name: no-name-dba5e72e-8cd9-4cd6-8b8e-70c2a91acb7a
[I 2022-12-15 18:50:58,790] Trial 0 finished with value: 0.7720157378336393 and parameters: {'lambda_l1': 2.02603704994206e-05, 'lambda_l2': 0.24790956599470865, 'num_leaves': 185, 'feature_fraction': 0.931809772602863, 'bagging_fraction': 0.5554747825790305, 'bagging_freq': 5, 'min_child_samples': 33}. Best is trial 0 with value: 0.7720157378336393.


[LightGBM] [Warning] feature_fraction is set=0.5620514552713423, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5620514552713423
[LightGBM] [Warning] lambda_l1 is set=0.9691114810152355, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.9691114810152355
[LightGBM] [Warning] bagging_fraction is set=0.6038150134948687, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6038150134948687
[LightGBM] [Warning] lambda_l2 is set=0.08751782841466708, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.08751782841466708
[LightGBM] [Warning] bagging_freq is set=7, subsample_freq=0 will be ignored. Current value: bagging_freq=7


[I 2022-12-15 18:50:59,863] Trial 1 finished with value: 0.7697706556805556 and parameters: {'lambda_l1': 0.9691114810152355, 'lambda_l2': 0.08751782841466708, 'num_leaves': 63, 'feature_fraction': 0.5620514552713423, 'bagging_fraction': 0.6038150134948687, 'bagging_freq': 7, 'min_child_samples': 44}. Best is trial 0 with value: 0.7720157378336393.


[LightGBM] [Warning] feature_fraction is set=0.6833710932120509, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6833710932120509
[LightGBM] [Warning] lambda_l1 is set=0.0017121352525293265, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.0017121352525293265
[LightGBM] [Warning] bagging_fraction is set=0.9176934233675403, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9176934233675403
[LightGBM] [Warning] lambda_l2 is set=0.7778849208240544, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.7778849208240544
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2


[I 2022-12-15 18:51:01,091] Trial 2 finished with value: 0.7585220423780756 and parameters: {'lambda_l1': 0.0017121352525293265, 'lambda_l2': 0.7778849208240544, 'num_leaves': 85, 'feature_fraction': 0.6833710932120509, 'bagging_fraction': 0.9176934233675403, 'bagging_freq': 2, 'min_child_samples': 49}. Best is trial 0 with value: 0.7720157378336393.


...

[LightGBM] [Warning] feature_fraction is set=0.6003320678352664, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6003320678352664
[LightGBM] [Warning] lambda_l1 is set=0.03959414247462695, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.03959414247462695
[LightGBM] [Warning] bagging_fraction is set=0.43915815815766396, subsample=1.0 will be ignored. Current value: bagging_fraction=0.43915815815766396
[LightGBM] [Warning] lambda_l2 is set=0.00010341823372380767, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.00010341823372380767
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1


[I 2022-12-15 18:52:10,789] Trial 98 finished with value: 0.7965850105559902 and parameters: {'lambda_l1': 0.03959414247462695, 'lambda_l2': 0.00010341823372380767, 'num_leaves': 7, 'feature_fraction': 0.6003320678352664, 'bagging_fraction': 0.43915815815766396, 'bagging_freq': 1, 'min_child_samples': 85}. Best is trial 89 with value: 0.8024360473313946.


[LightGBM] [Warning] feature_fraction is set=0.552484638805519, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.552484638805519
[LightGBM] [Warning] lambda_l1 is set=3.5502822366859292, reg_alpha=0.0 will be ignored. Current value: lambda_l1=3.5502822366859292
[LightGBM] [Warning] bagging_fraction is set=0.5153626786540089, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5153626786540089
[LightGBM] [Warning] lambda_l2 is set=0.0006313714689846409, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.0006313714689846409
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2


[I 2022-12-15 18:52:11,502] Trial 99 finished with value: 0.7839031166201903 and parameters: {'lambda_l1': 3.5502822366859292, 'lambda_l2': 0.0006313714689846409, 'num_leaves': 32, 'feature_fraction': 0.552484638805519, 'bagging_fraction': 0.5153626786540089, 'bagging_freq': 2, 'min_child_samples': 83}. Best is trial 89 with value: 0.8024360473313946.


Number of finished trials: 100
Best trial:
  Value: 0.8024360473313946
  Params: 
    lambda_l1: 0.20294875538671048
    lambda_l2: 1.641420137329384e-08
    num_leaves: 2
    feature_fraction: 0.6223600146497766
    bagging_fraction: 0.4400452648019314
    bagging_freq: 3
    min_child_samples: 78


In [48]:
lgb_best = study.best_params
lgb_best

{'lambda_l1': 0.20294875538671048,
 'lambda_l2': 1.641420137329384e-08,
 'num_leaves': 2,
 'feature_fraction': 0.6223600146497766,
 'bagging_fraction': 0.4400452648019314,
 'bagging_freq': 3,
 'min_child_samples': 78}

### Stacking Ensemble

In [49]:

ctb_best = {'learning_rate': 0.015778207856230707,
             'bagging_temperature': 0.1872474191063101,
             'n_estimators': 5431,
             'max_depth': 11,
             'random_strength': 2,
             'colsample_bylevel': 0.4416172828184351,
             'l2_leaf_reg': 0.0006604407155321971,
             'min_child_samples': 24,
             'max_bin': 206,
             'od_type': 'Iter'}

rf_best = {'max_depth': 16}

lgb_best = {'lambda_l1': 0.20294875538671048,
             'lambda_l2': 1.641420137329384e-08,
             'num_leaves': 2,
             'feature_fraction': 0.6223600146497766,
             'bagging_fraction': 0.4400452648019314,
             'bagging_freq': 3,
             'min_child_samples': 78}

In [ ]:
from sklearn.linear_model import LogisticRegression

# get a stacking ensemble of models
def get_stacking():
	# define the base models
	level0 = list()
	level0.append(('cat', CatBoostClassifier(**ctb_best)))
	level0.append(('rf', sklearn.ensemble.RandomForestClassifier(**rf_best)))
	level0.append(('lgb', lgb.LGBMClassifier(**lgb_best)))
	# define meta learner model
	level1 = LogisticRegression(max_iter=4000)
	# define the stacking ensemble
	model = StackingClassifier(estimators=level0, final_estimator=level1, cv=5)
	return model
 
# get a list of models to evaluate
def get_models():
	models = dict()
	#models['svc'] = CatBoostClassifier(**ctb_best)
	#models['rf'] = sklearn.ensemble.RandomForestClassifier(**rf_best)
	#models['lgb'] = lgb.LGBMClassifier(**lgb_best)
	models['stacking'] = get_stacking()
	return models
 
# evaluate a give model using cross-validation
def evaluate_model(model, x, y):
	cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
	scores = cross_val_score(model, x, y, scoring='neg_log_loss', cv=cv, n_jobs=-1)
	return scores
 
# get the models to evaluate
models = get_models()

# evaluate the models and store results
results, names = list(), list()
for name, model in models.items():
	scores = evaluate_model(model, x, y)
	results.append(scores)
	names.append(name)
	print('>%s %.3f (%.3f)' % (name, np.mean(scores), np.std(scores)))


# plot model performance for comparison
pyplot.boxplot(results, labels=names, showmeans=True)
pyplot.show()